In [2]:
import os
import numpy as np
directory = './solvation_csv'
out_files = np.array([f for f in os.listdir(directory) if f.endswith(".pkl")])
numbers = []
'''
for file_name in out_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'r') as file:
        last_line = file.readlines()[-1].strip()
        words = last_line.split()
        for word in words:
            try:
                number = float(word)
                numbers.append(number)
            except:
                continue
'''
print(len(out_files))
print(np.mean(np.array(numbers)))

788
nan


/nas/longleaf/home/rdey/anaconda3/envs/implicit/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/nas/longleaf/home/rdey/anaconda3/envs/implicit/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [48]:
import pandas as pd
import pickle as pkl
import numpy as np
pkl_files = np.array([f for f in os.listdir(directory) if f.endswith(".pkl")])
dataframes = []
sorted_pkl_files = sorted(pkl_files,  key=lambda s: int(''.join(filter(str.isdigit, s))))

for file_name in sorted_pkl_files:
    file_path = os.path.join(directory, file_name)
    try:
        with open(file_path, 'rb') as file:
            df = pkl.load(file)
            dataframes+=df
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

    

In [56]:
with open("solvation_final.pkl", 'wb') as file:
    pkl.dump(dataframes, file)


In [1]:
import torch
import torch.cuda
import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pickle as pkl
from tqdm import tqdm
import numpy as np
from torchmdnet.models.tensornet import TensorNet

torch.cuda.is_available == True

ModuleNotFoundError: No module named 'torchmdnet'

In [2]:
input = QM9("")
data2 = []
dataset = []

with open("solvation_final.pkl", 'rb') as file:
    data = pkl.load(file)

for force in tqdm(data):
    if hasattr(force, '_value'):  
        force = force._value
        if len(force) < 30:
            padding = np.zeros((30-len(force), force.shape[1]))
        force = torch.tensor(force)
    data2.append(force)
        

for i in tqdm(input):
    idx = int(i.name.split('_')[-1]) - 1
    if not isinstance(data2[idx], type(-1)):
        if(len(data[idx]) == len(i.z)):
            i.y = data2[idx]
            dataset.append(i)
    


100%|██████████| 130831/130831 [00:18<00:00, 7075.07it/s]


In [5]:
test_dataset = dataset[:10000]
val_dataset = dataset[10000:20000]
train_dataset = dataset[20000:]
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

len(test_dataset)

1000

In [78]:

model = TensorNet(hidden_channels=128)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
c_loss = torch.nn.MSELoss()

dataset[0].z

tensor([6, 1, 1, 1, 1])

In [103]:
#CPU
for epoch in range(10):
    print(f"Current Epoch:{epoch}")
    model.train()
    running_loss = 0.0
    print("Running Training Batches")
    for batch in tqdm(train_loader):
        args= batch.to_dict()
        y_true = args['y'].float()
        y = model(args['z'], args['pos'], args['batch'])
        optimizer.zero_grad()
        y_pred = y[3].requires_grad_()
        loss = c_loss(y_pred, y_true)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_train_loss = running_loss / len(train_loader)

    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        print("Running Validation Batches")
        for batch in tqdm(val_loader):
            args = batch.to_dict()
            y_true = args['y'].float()
            y = model(args['z'], args['pos'], args['batch'])
            y_pred = y[3].requires_grad_()
            val_loss = c_loss(y_pred, y_true)
            val_running_loss += val_loss.item()

    epoch_val_loss = val_running_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{10}], Training Loss: {epoch_train_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}")

print("Finished")




Current Epoch:0
Running Training Batches


  1%|          | 4/619 [00:11<29:49,  2.91s/it]


KeyboardInterrupt: 

In [101]:
type(y[3])

torch.Tensor

In [99]:
#GPU CUDA
model = model.cuda()
c_loss = c_loss.cuda()

for epoch in range(10):
    print(f"Current Epoch:{epoch}")
    model.train()
    running_loss = 0.0
    print("Running Training Batches")
    for batch in tqdm(train_loader):
        args = batch.to_dict()
        y_true = args['y'].float().cuda()  # Move y_true to CUDA
        z = args['z'].cuda()  # Move z to CUDA
        pos = args['pos'].cuda()  # Move pos to CUDA
        batch_idx = args['batch'].cuda()  # Move batch_idx to CUDA
        
        y = model(z, pos, batch_idx)
        optimizer.zero_grad()
        y_pred = y[3].cuda().requires_grad_()  # Move y_pred to CUDA and ensure gradients are enabled
        loss = c_loss(y_pred, y_true)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_train_loss = running_loss / len(train_loader)

    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        print("Running Validation Batches")
        for batch in tqdm(val_loader):
            args = batch.to_dict()
            y_true = args['y'].float().cuda()  # Move y_true to CUDA
            z = args['z'].cuda()  # Move z to CUDA
            pos = args['pos'].cuda()  # Move pos to CUDA
            batch_idx = args['batch'].cuda()  # Move batch_idx to CUDA
            
            y = model(z, pos, batch_idx)
            y_pred = y[3].cuda().requires_grad_()  # Move y_pred to CUDA and ensure gradients are enabled
            val_loss = c_loss(y_pred, y_true)
            val_running_loss += val_loss.item()

    epoch_val_loss = val_running_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{10}], Training Loss: {epoch_train_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}")

print("Finished")


AssertionError: Torch not compiled with CUDA enabled

In [60]:


model.eval()
test_running_loss = 0.0
total_error = 0.0
with torch.no_grad():
    for batch in tqdm(test_loader):
        print("Running Test Batches")
        args = batch.to_dict()
        
        z = args['z']
        pos = args['pos']
        batch_idx = args['batch']
        y_true = args['y'].float()
        
        y = model(z, pos, batch_idx)
        test_loss = c_loss(y[0], y_true)
        test_running_loss += test_loss.item()
        for idx, Y in enumerate(y_true):
            Y_true = torch.sum(Y)
            Y = torch.sum(y[0][idx])
            error = int((Y_true-Y)/Y_true)
            total_error += error

print(total_error/len(test_loader))




 25%|██▌       | 2/8 [00:00<00:00, 15.66it/s]

Running Test Batches
Running Test Batches
Running Test Batches


 50%|█████     | 4/8 [00:00<00:00, 14.20it/s]

Running Test Batches
Running Test Batches
Running Test Batches


100%|██████████| 8/8 [00:00<00:00, 14.46it/s]

Running Test Batches
Running Test Batches
-3722.75


In [61]:
y

(tensor([[-0.0484,  0.0659, -0.1074],
         [-0.2727, 24.0103, -0.2776],
         [ 2.6303,  8.9091,  1.9426],
         ...,
         [-0.0975,  1.1149, -0.1226],
         [-0.2631, 24.3339, -0.2783],
         [-0.2785, 24.4465, -0.2783]]),
 None,
 tensor([7, 6, 6,  ..., 1, 1, 1]),
 tensor([[ 1.0690e-01, -1.1525e+00, -5.1000e-03],
         [ 2.5100e-02,  3.6000e-03,  2.5000e-03],
         [-6.4400e-02,  1.4102e+00,  1.1600e-02],
         ...,
         [ 1.6700e+00, -1.9860e-01,  1.3692e+00],
         [-3.4811e+00, -1.5101e+00, -2.4590e-01],
         [-3.6328e+00, -1.4349e+00, -1.9609e+00]]),
 tensor([  0,   0,   0,  ..., 103, 103, 103]))